In [2]:
pip install tensorflow transformers


In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import RobertaTokenizer

# Load your dataset
df = pd.read_csv("/content/sentiment_analysis_self_driving_vehicles.csv")
df_shuffled = df.sample(frac=1).reset_index(drop=True)
number_of_clients = 3  # Replace with your number of clients

# Calculate the size of each chunk
chunk_size = len(df_shuffled) // number_of_clients
i = 0
# Split the DataFrame into chunks
df_client = df.iloc[0:chunk_size]


# Preprocessing
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def encode_reviews(tokenizer, reviews, max_length):
    return tokenizer.batch_encode_plus(reviews,
                                       truncation=True,
                                       padding='max_length',
                                       max_length=max_length,
                                       return_attention_mask=True,
                                       return_token_type_ids=False)

# Encode the text
encoded_data = encode_reviews(tokenizer, df_client['Text'].tolist(), max_length=128)

# Create inputs
inputs = {
    'input_ids': encoded_data['input_ids'],
    'attention_mask': encoded_data['attention_mask']
}

# Convert labels to numerical format
label_dict = {"Positive": 1, "Negative": 0, "Neutral": 2}  # Adjust as needed
df_client['Sentiment'] = df_client['Sentiment'].replace(label_dict)

# Split the dataset
train_input_ids, test_input_ids, train_attention_masks, test_attention_masks, train_labels, test_labels = train_test_split(
    encoded_data['input_ids'], encoded_data['attention_mask'], df_client['Sentiment'].values, test_size=0.2)

# Reconstruct the inputs as dictionaries
train_inputs = {'input_ids': train_input_ids, 'attention_mask': train_attention_masks}
test_inputs = {'input_ids': test_input_ids, 'attention_mask': test_attention_masks}


<ipython-input-7-9c09f210c533>:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_client['Sentiment'] = df_client['Sentiment'].replace(label_dict)


In [8]:
import tensorflow as tf

def create_tf_dataset(inputs, labels, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
    dataset = dataset.batch(batch_size)
    return dataset

batch_size = 32

train_dataset = create_tf_dataset(train_inputs, train_labels, batch_size)
test_dataset = create_tf_dataset(test_inputs, test_labels, batch_size)


In [9]:
from transformers import TFRobertaForSequenceClassification
from transformers import AdamW
import tensorflow as tf
# Load the RoBERTa model
client_model1 = TFRobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(label_dict))

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
client_model1.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [ ]:
epochs = 3

client_model1.fit(train_dataset,
          epochs=epochs,
          validation_data=test_dataset)


Epoch 1/3
1/5 [=====>........................] - ETA: 4:00 - loss: 1.0666 - accuracy: 0.5000

In [ ]:
client_model1.evaluate(test_dataset)


In [ ]:
#client2

In [ ]:
i = 1
# Split the DataFrame into chunks
df_client = df.iloc[chunk_size:2*chunk_size]


# Preprocessing
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def encode_reviews(tokenizer, reviews, max_length):
    return tokenizer.batch_encode_plus(reviews,
                                       truncation=True,
                                       padding='max_length',
                                       max_length=max_length,
                                       return_attention_mask=True,
                                       return_token_type_ids=False)

# Encode the text
encoded_data = encode_reviews(tokenizer, df_client['Text'].tolist(), max_length=128)

# Create inputs
inputs = {
    'input_ids': encoded_data['input_ids'],
    'attention_mask': encoded_data['attention_mask']
}

# Convert labels to numerical format
label_dict = {"Positive": 1, "Negative": 0, "Neutral": 2}  # Adjust as needed
df_client['Sentiment'] = df_client['Sentiment'].replace(label_dict)

# Split the dataset
train_input_ids, test_input_ids, train_attention_masks, test_attention_masks, train_labels, test_labels = train_test_split(
    encoded_data['input_ids'], encoded_data['attention_mask'], df_client['Sentiment'].values, test_size=0.2)

# Reconstruct the inputs as dictionaries
train_inputs = {'input_ids': train_input_ids, 'attention_mask': train_attention_masks}
test_inputs = {'input_ids': test_input_ids, 'attention_mask': test_attention_masks}


In [ ]:
import tensorflow as tf

def create_tf_dataset(inputs, labels, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
    dataset = dataset.batch(batch_size)
    return dataset

batch_size = 32

train_dataset = create_tf_dataset(train_inputs, train_labels, batch_size)
test_dataset = create_tf_dataset(test_inputs, test_labels, batch_size)
from transformers import TFRobertaForSequenceClassification
from transformers import AdamW
import tensorflow as tf
# Load the RoBERTa model
client_model2 = TFRobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(label_dict))

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
client_model2.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
epochs = 3

client_model2.fit(train_dataset,
          epochs=epochs,
          validation_data=test_dataset)
client_model2.evaluate(test_dataset)


In [ ]:
#client 3

In [ ]:
i = 2
# Split the DataFrame into chunks
df_client = df.iloc[2*chunk_size:]


# Preprocessing
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def encode_reviews(tokenizer, reviews, max_length):
    return tokenizer.batch_encode_plus(reviews,
                                       truncation=True,
                                       padding='max_length',
                                       max_length=max_length,
                                       return_attention_mask=True,
                                       return_token_type_ids=False)

# Encode the text
encoded_data = encode_reviews(tokenizer, df_client['Text'].tolist(), max_length=128)

# Create inputs
inputs = {
    'input_ids': encoded_data['input_ids'],
    'attention_mask': encoded_data['attention_mask']
}

# Convert labels to numerical format
label_dict = {"Positive": 1, "Negative": 0, "Neutral": 2}  # Adjust as needed
df_client['Sentiment'] = df_client['Sentiment'].replace(label_dict)

# Split the dataset
train_input_ids, test_input_ids, train_attention_masks, test_attention_masks, train_labels, test_labels = train_test_split(
    encoded_data['input_ids'], encoded_data['attention_mask'], df_client['Sentiment'].values, test_size=0.2)

# Reconstruct the inputs as dictionaries
train_inputs = {'input_ids': train_input_ids, 'attention_mask': train_attention_masks}
test_inputs = {'input_ids': test_input_ids, 'attention_mask': test_attention_masks}


In [ ]:
import tensorflow as tf

def create_tf_dataset(inputs, labels, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
    dataset = dataset.batch(batch_size)
    return dataset

batch_size = 32

train_dataset = create_tf_dataset(train_inputs, train_labels, batch_size)
test_dataset = create_tf_dataset(test_inputs, test_labels, batch_size)
from transformers import TFRobertaForSequenceClassification
from transformers import AdamW
import tensorflow as tf
# Load the RoBERTa model
client_model3= TFRobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(label_dict))

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
client_model3.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
epochs = 3

client_model3.fit(train_dataset,
          epochs=epochs,
          validation_data=test_dataset)
client_model3.evaluate(test_dataset)

#get model paramters

In [ ]:
weights = []
weights.append(client_model1.get_weights())
weights.append(client_model2.get_weights())
weights.append(client_model3.get_weights())

In [ ]:
from transformers import GPT2Model

# Initialize the model architecture
from transformers import BertModel, BertTokenizer

# Initialize the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertModel.from_pretrained('bert-base-cased')
import torch
avg_params = [sum(param) / len(param) for param in zip(*weights)]
model.set_weights( avg_params)

# Path to your custom weights


In [ ]:
model.evaluate(test_dataset)
